# Cas12 Screening for self-processing pre-crRNAs v1.0

`Input`: Cas12 proteins in .fasta/.faa format.

`Output`: predicted wether the Cas12 candidates would self-process their pre-crRNAs.

*Note: before executing the following commands, please ensure that you have to save/add the following folders into your Google Drive ("MyDrive" folder):

- [model](https://drive.google.com/drive/folders/1y4WKwsoBsqBb_R2Cdj0cwYiLIPnBXj01?usp=sharing)

- [inputs](https://drive.google.com/drive/folders/18GGlIEWYtJVTn2oBXMqbghyYQCLKelLg?usp=sharing)

In [ ]:
#@title Step.01 setup **Environment** (~2m)
%%time
import os, time, signal
import sys, random, string, re
## mount google drive
from google.colab import drive
drive.mount('/content/drive')

## install conda
!pip install -q condacolab
import condacolab
condacolab.install()
!conda update -n base -c conda-forge conda -y

## packages install
!pip install biopython
!pip install git+https://github.com/facebookresearch/esm.git
!conda install -c conda-forge pytorch_geometric -y
!conda install -c conda-forge biotite -y

In [ ]:
################### Models ###################

import os, re
import numpy as np
import pandas as pd
from transformers import EsmModel, AutoTokenizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.backends.cudnn as cudnn
import glob
import esm
import Bio.PDB
import random
import argparse

## Unet model
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(mid_channels),
            #nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(mid_channels),
            #nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

class UNet(nn.Module):
    def __init__(self, n_channels, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        #self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 32)
        self.down1 = Down(32, 64)
        self.down2 = Down(64, 128)
        self.down3 = Down(128, 256)
        factor = 2 if bilinear else 1
        self.down4 = Down(256, 512 // factor)
        self.up1 = Up(512, 256 // factor, bilinear)
        self.up2 = Up(256, 128 // factor, bilinear)
        self.up3 = Up(128, 64 // factor, bilinear)
        self.up4 = Up(64, 32, bilinear)

        self.softmax = nn.Softmax(dim=1)
        self.conv2 = nn.Conv2d(32, 2, kernel_size=1,bias=False)

    def forward(self, x):
        # print(x.size())
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        # se模块

        x = self.conv2(x)
        return x

## Models

class ModelLasthidden(nn.Module):     ## for CLS classification
    def __init__(self, args):
        super(ModelLasthidden, self).__init__()

        self.dense = nn.Linear(args.emb_dim, args.emb_dim)
        self.out_proj = nn.Linear(args.emb_dim, 2)
        self.dropout = nn.Dropout(args.dropout)

    def forward(self, x):

        x = x[:, :, 0, :]
        x = x.squeeze(1)
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        y = self.out_proj(x)

        return y

class ModelLastHiddenMatrix(nn.Module):     ## for embedding classification
    def __init__(self, args):
        super(ModelLastHiddenMatrix, self).__init__()
        self.unet = UNet(1)
        self.am_pool = nn.AdaptiveMaxPool2d((16,16))
        self.fc1 = nn.Linear(256*2, 64)
        self.fc2 = nn.Linear(64,2)
        self.dropout = nn.Dropout(args.dropout)

    def forward(self, x):

        x = x[:,:,1:-1,:]
        x = self.unet(x)
        x = self.am_pool(x)
        x = x.view(x.size(0),-1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        y = self.fc2(x)
        return y

class ModelDist(nn.Module):     ## for distance map or esm_if classification
    def __init__(self, args):
        super(ModelDist, self).__init__()
        self.unet = UNet(1)
        self.am_pool = nn.AdaptiveMaxPool2d((16,16))
        self.fc1 = nn.Linear(256*2, 64)
        self.fc2 = nn.Linear(64,2)
        self.dropout = nn.Dropout(args.dropout)

    def forward(self, x):

        x = self.unet(x)
        x = self.am_pool(x)
        x = x.view(x.size(0),-1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        y = self.fc2(x)
        return y

class MyDatasetEmbDist(Dataset):

    def __init__(self, fea_path,datatype='embedding'):
        self.fea_path = fea_path
        self.file_list = sorted(glob.glob(fea_path+'/*.npy'))
        self.datatype = datatype

    def __getitem__(self, index):

        name = os.path.basename(self.file_list[index])
        if self.datatype == 'embedding':
          feature = torch.from_numpy(np.load(os.path.join(self.fea_path, name))).float()
        elif self.datatype == 'structure':
          feature = np.load(os.path.join(self.fea_path, name))
          feature = torch.unsqueeze(torch.from_numpy(feature).float(), 0)
        return feature, name[:-4]

    def __len__(self):

        return len(self.file_list)

In [ ]:
################### Data Processing ###################

def data_gener_esm(layer,model,outdir,indir=None):
    output_path = os.path.join(outdir,'the_%d_layer' % layer)
    try:
        os.makedirs(output_path)
    except:
        print('Output directory existed: %s' % output_path)
    if indir==None:
        file_list = glob.glob('protein'+'/*.faa')
    else:
        file_list = glob.glob(indir+'/*.faa')
    tokenizer = AutoTokenizer.from_pretrained(model)
    model = EsmModel.from_pretrained(model).to('cuda')

    for f in file_list:
        lines = open(f, 'r').readlines()
        for i in range(len(lines)):
            if lines[i][0] == '>':
                seq = lines[i+1].strip()
                inputs = tokenizer(seq, return_tensors="pt").to('cuda')
                outputs = model(output_hidden_states=True,return_dict=True,**inputs)
                this_hidden_states = outputs.hidden_states[layer]
                np.save(os.path.join(output_path, lines[i].strip()[1:]), this_hidden_states.detach().cpu().numpy())

def calc_residue_dist(residue_one, residue_two,ca='CA') :
    """Returns the C-alpha distance between two residues"""
    diff_vector  = residue_one[ca].coord - residue_two[ca].coord
    return np.sqrt(np.sum(diff_vector * diff_vector))

def calc_dist_matrix(chain_one, chain_two,ca='CA') :
    """Returns a matrix of C-alpha distances between two chains"""
    answer = np.zeros((len(chain_one), len(chain_two)),'float')
    for row, residue_one in enumerate(chain_one) :
        for col, residue_two in enumerate(chain_two) :
            answer[row, col] = calc_residue_dist(residue_one, residue_two, ca)
    return answer

def data_gener_from_pdb(outdir,mode='esmif',indir=None):
    ## output directory
    try:
        os.makedirs(outdir)
    except:
        print('Output directory existed: %s' % outdir)

    ## input pdbs
    if indir==None:
        pdb_list = glob.glob('protein'+'/*.pdb')
    else:
        pdb_list = glob.glob(indir+'/*.pdb')

    ## esmfold_if
    if mode == 'esmif':
        ## load esmfold_if
        model, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()   ## convert pdb structure to embedding (dim=512)
        model = model.eval().cuda()
        for this_pdb in pdb_list:
            name = re.sub('\.pdb$','',os.path.basename(this_pdb))
            ## converting
            chain_id = 'A'
            structure = esm.inverse_folding.util.load_structure(this_pdb, chain_id)
            coords, native_seq = esm.inverse_folding.util.extract_coords_from_structure(structure)
            rep = esm.inverse_folding.util.get_encoder_output(model, alphabet, coords)
            ## save
            np.save(os.path.join(outdir,name),rep.cpu().detach().numpy())
    ## distance map
    elif mode == 'distance':
        for this_pdb in pdb_list:
            name = re.sub('\.pdb$','',os.path.basename(this_pdb))
            structure = Bio.PDB.PDBParser().get_structure(name, this_pdb)
            model = structure[0]
            dist_matrix = calc_dist_matrix(model["A"], model["A"],ca='CA')
            np.save(os.path.join(outdir,name),dist_matrix)

device = torch.device("cuda")

In [ ]:
#@title Step.02 run **Self-processing Prediction** (~1m)
%%time
#@markdown **Parameters** settings
#@markdown ---
args = argparse.ArgumentParser(description='SelfProcess')
datatype = 'embedding'
model = "CLS_best" #@param ["CLS_best","CLS_lastlayer","Embedding_best","ESM_inverfold","DistanceMap"] {type:"string"}
if model == 'CLS_best':
  _model = 'ModelLasthidden'
  _test_model = 'drive/MyDrive/models/spCas/esm2_650M_10th_cls.pt'
elif model == 'CLS_lastlayer':
  _test_model = 'drive/MyDrive/models/spCas/esm2_650M_last_cls.pt'
elif model == 'Embedding_best':
  _model = 'ModelLastHiddenMatrix'
  _test_model = 'drive/MyDrive/models/spCas/esm2_650M_23th_emb.pt'
elif model == 'ESM_inverfold':
  _model = 'ModelDist'
  _test_model = 'drive/MyDrive/models/spCas/esm_if.pt'
  datatype = 'structure'
elif model == 'DistanceMap':
  _model = 'ModelDist'
  _test_model = 'drive/MyDrive/models/spCas/distance_map.pt'
  datatype = 'structure'

args.model = _model
args.test_model = _test_model

feature_path = 'features'
args.fea_path = feature_path
try:
  os.makedirs(args.fea_path)
except:
  pass

from google.colab import files
input_faa = 'drive/MyDrive/inputs/suspicious.faa' #@param {type:"string"}
#@markdown - input protein sequence `.faa` for "CLS_best", "CLS_lastlayer", and "Embedding_best" modles.
#@markdown - or input your own `.faa` file.
input_pdb = 'drive/MyDrive/inputs/suspicious.pdb' #@param {type:"string"}
#@markdown - input protein sequence `.pdb` for "ESM_inverfold" and "DistanceMap" modles.
#@markdown - or input your own `.pdb` file.
emb_dim = 1280 #@param ["1280"] {type:"raw"}
#@markdown - embedding size for ESM-2 `esm2_t33_650M_UR50D`.
seed = 42 #@param ["42"] {type:"raw"}
#@markdown - ramdom seed. default `42`.
out_table = 'sp_result.csv' #@param {type:"string"}
#@markdown - output table of prediction.

args.emb_dim = emb_dim
args.seed = seed
args.out_tab = out_table
args.dropout = 0.2
if model=='CLS_best' or model=='CLS_lastlayer' or model=='Embedding_best':
  if input_faa:
    !cp {input_faa} ./
  else:
    upload = files.upload()
    input_faa = list(upload.keys())[0]
  if model=='CLS_best':
    data_gener_esm(10,'facebook/esm2_t33_650M_UR50D',args.fea_path,'./')
    args.fea_path = args.fea_path + '/the_10_layer'
    trained_model = ModelLasthidden(args).to(device)
  elif model=='CLS_lastlayer':
    data_gener_esm(33,'facebook/esm2_t33_650M_UR50D',args.fea_path,'./')
    args.fea_path = args.fea_path + '/the_33_layer'
    trained_model = ModelLasthidden(args).to(device)
  elif model=='Embedding_best':
    data_gener_esm(23,'facebook/esm2_t33_650M_UR50D',args.fea_path,'./')
    args.fea_path = args.fea_path + '/the_23_layer'
    trained_model = ModelLastHiddenMatrix(args).to(device)
else:
  if input_pdb:
    !cp {input_pdb} ./
  else:
    upload = files.upload()
    input_pdb = list(upload.keys())[0]
  trained_model = ModelDist(args).to(device)
  if model=='ESM_inverfold':
    data_gener_from_pdb(args.fea_path,'esmif','./')
  elif model=='DistanceMap':
    data_gener_from_pdb(args.fea_path,'distance','./')

## Prediciton
trained_model.load_state_dict(torch.load(args.test_model), strict=False)
test_set = MyDatasetEmbDist(args.fea_path,'embedding' )
test_loader = DataLoader(test_set, 1, shuffle=None, num_workers=1)
trained_model.eval().cuda()

preds, probs, names = [], [], []
with torch.no_grad():
  for fea, name in test_loader:
    fea = fea.to(device)
    layer = nn.Softmax(dim=1)
    output = layer(trained_model(fea))
    preds.append(torch.argmax(output,dim=1).cpu().numpy())
    probs.append(output[0][1].item())
    names.append(name)

data = {
"names":[item[0] for item in names],
"preds":[item[0] for item in preds],
"probs":probs
}

df = pd.DataFrame(data)
df.to_csv(os.path.join(out_table),sep='\t',index=False)

In [ ]:
df[['names','preds','probs']]

In [ ]:
#@title Step.03 Package and download results
from google.colab import files
!zip -r CasPrediction.zip {out_table}
files.download(f"spCasPrediction.zip")

# *Note*

This pipeline is designed to screen for Cas12 candidates, identifying effectors capable of self-processing their own pre-crRNAs.

We offer five modes of prediction:

- `CLS_best`. Utilizes the 10th layer output CLS as the input to a classifier.

- `CLS_lastlayer`. Utilized the last layer output CLS as the input to a classifier.

- `Embedding_best`. Utilizes the 23th layer output embedding as the input to a classifier.

- `ESM_inverfold`. Utilized the inverse-folding embedding of a structure `.pdb` as the input to a classifier.

- `DistanceMap`. Utilized the distance map matrices of a structure `.pdb` as the input to a classifier.


---

The output table `sp_result.csv` presents the following columns:

- `names`. The ids of each protein.

- `preds`. Model predicted labels.

- `probs`. The normalized probability of this protein to be identified as self-processing its own pre-crRNAs.
